In [ ]:
from PIL import Image

import torch
import Imath
import OpenEXR
import numpy as np
import cv2
import time
import random

In [ ]:
def load_exr(filename):
    """Load an EXR image and return it as a NumPy array."""
    exr_file = OpenEXR.InputFile(filename)
    header = exr_file.header()
    dw = header['dataWindow']
    width = dw.max.x - dw.min.x + 1
    height = dw.max.y - dw.min.y + 1

    channels = list(header['channels'].keys())  # Convert to a list
    pixel_type = Imath.PixelType(Imath.PixelType.FLOAT)

    # Read channels
    img = {ch: np.frombuffer(exr_file.channel(ch, pixel_type), dtype=np.float32)
           .reshape(height, width) for ch in channels}

    return img, width, height, channels

def resize_exr(img, new_width, new_height):
    """Resize an EXR image using OpenCV."""
    resized_img = {ch: cv2.resize(img[ch], (new_width, new_height), interpolation=cv2.INTER_NEAREST_EXACT)
                   for ch in img}
    return resized_img

def luminance(rgb):
    return rgb @ np.asarray((0.2126, 0.7152, 0.0722), dtype=np.float32)

def change_luminance(rgb: np.ndarray, l_out: np.ndarray):
    l_in: np.ndarray = luminance(rgb)
    return rgb * np.expand_dims((l_out / l_in), axis = -1)

def rgb_to_ext_reinhard(rgb, max_white_l=100):
    l_old = luminance(rgb)
    numerator = l_old * (1.0 + (l_old / (max_white_l * max_white_l)))
    l_new = numerator / (1.0 + l_old)

    return change_luminance(rgb, l_new)

def reinhard_to_rgb(reinhard, white_lum=100):
    new_lum = luminance(reinhard)

    # Solve the quadratic equation
    a = 1.0 / white_lum ** 2
    b = 1.0 - new_lum
    c = -new_lum

    discriminant = np.sqrt(b ** 2 - 4 * a * c)
    lum = (-b + discriminant) / (2 * a)

    # Scale reinhard_rgb back to original rgb
    return reinhard * np.where(new_lum != 0, (lum / new_lum), 0)[:, :, None]

def rgb_to_hlg(rgb):
    hlg = None
    with np.errstate(divide='ignore', invalid='ignore'):
        hlg = np.where(rgb <= 1.0,
                    0.5 * np.sqrt(rgb),
                    0.17883277 * np.log(rgb - 0.28466892) + 0.55991073)
        hlg[np.isinf(hlg)] = 0
        hlg[np.isnan(hlg)] = 0
    return hlg


def hlg_to_rgb(hlg):
    rgb = None
    with np.errstate(divide='ignore', invalid='ignore'):
        rgb = np.where(hlg <= 0.5,
                    np.square(2.0 * hlg),
                    np.exp((hlg - 0.55991073) / 0.17883277) + 0.28466892)
        rgb[np.isinf(rgb)] = 0
        rgb[np.isnan(rgb)] = 0
    return rgb

def pq_to_rgb(rgb: np.ndarray, mul: int = 10000) -> np.ndarray:
    m1 = 0.1593017578125
    m2 = 78.84375
    c1 = 0.8359375
    c2 = 18.8515625
    c3 = 18.6875

    ret = None
    with np.errstate(divide='ignore', invalid='ignore'):
        E_p = rgb ** (1/m2)
        par = np.maximum((E_p - c1), 0) / (c2 - c3 * E_p)
        ret = mul * (par ** (1/m1))
        ret[np.isinf(ret)] = 0
        ret[np.isnan(ret)] = 0
    
    return np.clip(ret, 0, 1)

def rgb_to_pq(pq: np.ndarray, div: int = 10000) -> np.ndarray:
    m1 = 0.1593017578125
    m2 = 78.84375
    c1 = 0.8359375
    c2 = 18.8515625
    c3 = 18.6875

    ret = None
    with np.errstate(divide='ignore', invalid='ignore'):
        Y = pq / div
        ret = ((c1 + c2 * (Y ** m1)) / (1 + c3 * (Y ** m1))) ** m2
        ret[np.isinf(ret)] = 0
        ret[np.isnan(ret)] = 0

    return ret

def save_exr(img, filename):
    height, width, _ = img.shape
    header = OpenEXR.Header(width, height)
    exr = OpenEXR.OutputFile(filename, header)

    r, g, b = np.split(img, 3, axis=-1)
    print(f"{r.shape}-{g.shape}-{b.shape}")
    exr.writePixels({'R': r.tobytes(),
	                 'G': g.tobytes(),
	                 'B': b.tobytes()})
    exr.close()

In [ ]:
import matplotlib.pyplot as plt
from diffusers import StableDiffusionInpaintPipeline

seed = int(time.time())
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    # revision="fp16",
    torch_dtype=torch.float16,
    use_safetensors = True,
    safety_checker=None
)

pipe.unet.load_attn_procs("/home/rhohen/Workspace/diffusion-env/scripts/finetune_lora_no_resizing/checkpoint-5000")
# pipe.unet.load_attn_procs("/home/rhohen/Workspace/diffusion-env/scripts/finetune_lora_no_resizing_normalize/checkpoint-5000")
# pipe.unet.load_attn_procs("/home/rhohen/Workspace/diffusion-env/scripts/finetune_lora_hlg/checkpoint-5000")
# pipe.unet.load_attn_procs("/home/rhohen/Workspace/diffusion-env/scripts/finetune_lora_pq/checkpoint-5000")
pipe.to('cuda')

#TODO: Cambiare la pipeline di diffusers, per far si che accetti una immagine direttamente floating point e la resitituisca tale

#### Example with 2 inpaintings (PQ)

In [ ]:
# prompt = 'Round Sunlight, strong high luminance'
# prompt = 'White Round Sun, Outdoor Environment map'
prompt = "Dark Sky, low luminance, High Resolution, Outdoor Environment map"
# prompt = "Remove Sun, Sky, no luminance, High Resolution, Outdoor Environment map"
negative_prompt = 'boxes, artifacts, light'

# image and mask_image should be PIL images.
#The mask structure is white for inpainting and black for keeping as is
# img, width, height, channels = load_exr("../data/abandoned_church_1k.exr")
img, width, height, channels = load_exr("./brick_factory_white.exr")
new_width, new_height = int(width * 1.0), int(height * 1.0)
resized_img = resize_exr(img, new_width, new_height)
resized_rgb_image = np.stack([resized_img['R'], resized_img['G'], resized_img['B']], axis=-1)
# rein_out = rgb_to_ext_reinhard(resized_rgb_image)
# rein_out = rgb_to_hlg(resized_rgb_image)
rein_out = rgb_to_pq(resized_rgb_image, div=100)
# inp_img = Image.fromarray((rein_out * 255).clip(0, 255).astype(np.uint8))
# plt.imshow(inp_img)

mask_image = Image.open("mask_1_sky_large.png")
mask_image = pipe.mask_processor.blur(mask_image, blur_factor=10)
# plt.imshow(mask_image)

image: Image.Image = pipe(prompt=prompt, negative_prompt=negative_prompt, image=rein_out, mask_image=mask_image, num_inference_steps=50, strength=0.7, guidance_scale=7.5, width=1024, height=512, output_type='np').images[0]
plt.imshow(image)

prompt = "Sunlight, Natural, High Resolution, Outdoor Environment map"
negative_prompt = 'boxes, artifacts'
mask_image = Image.open("mask_2_light_large.png")
mask_image = pipe.mask_processor.blur(mask_image, blur_factor=10)
image: Image.Image = pipe(prompt=prompt, negative_prompt=negative_prompt, image=image, mask_image=mask_image, num_inference_steps=100, strength=0.7, guidance_scale=7.5, width=1024, height=512, output_type='np').images[0]

# rescaled_image = np.array(image, dtype = np.float32) / 255.0
rescaled_image = image
save_exr(pq_to_rgb(rescaled_image, mul=100), 'Output_int.exr')
rescaled_image = Image.fromarray((image * 255).clip(0, 255).astype(np.uint8))
rescaled_image.save("Output_int.png")

#### Example with 2 inpaintings (Reinhard)

In [ ]:
# prompt = 'Round Sunlight, strong high luminance'
# prompt = 'White Round Sun, Outdoor Environment map'
prompt = "Dark Sky, no luminance, High Resolution, Outdoor Environment map"
# prompt = "Remove Sun, Sky, no luminance, High Resolution, Outdoor Environment map"
negative_prompt = 'boxes, artifacts, light'

# image and mask_image should be PIL images.
#The mask structure is white for inpainting and black for keeping as is
# img, width, height, channels = load_exr("../data/abandoned_church_1k.exr")
img, width, height, channels = load_exr("./brick_factory_white.exr")
new_width, new_height = int(width * 1.0), int(height * 1.0)
resized_img = resize_exr(img, new_width, new_height)
resized_rgb_image = np.stack([resized_img['R'], resized_img['G'], resized_img['B']], axis=-1)
rein_out = rgb_to_ext_reinhard(resized_rgb_image)
inp_img = rein_out
# inp_img = Image.fromarray((rein_out * 255).clip(0, 255).astype(np.uint8))
# plt.imshow(inp_img)

mask_image = Image.open("mask_1_sky_large.png")
mask_image = pipe.mask_processor.blur(mask_image, blur_factor=10)
# plt.imshow(mask_image)

image: Image.Image = pipe(prompt=prompt, negative_prompt=negative_prompt, image=inp_img, mask_image=mask_image, num_inference_steps=50, strength=0.7, guidance_scale=7.5, width=1024, height=512, output_type='np').images[0]
# plt.imshow(image)

prompt = "Sunlight, Natural, High Resolution, Outdoor Environment map"
negative_prompt = 'boxes, artifacts'
mask_image = Image.open("mask_2_light_large.png")
mask_image = pipe.mask_processor.blur(mask_image, blur_factor=10)
image: Image.Image = pipe(prompt=prompt, negative_prompt=negative_prompt, image=image, mask_image=mask_image, num_inference_steps=200, strength=0.6, guidance_scale=7.5, width=1024, height=512, output_type='np').images[0]

# rescaled_image = np.array(image, dtype = np.float32) / 255.0
rescaled_image = image
save_exr(reinhard_to_rgb(rescaled_image, white_lum=100), 'Output_int.exr')
rescaled_image = Image.fromarray((image * 255).clip(0, 255).astype(np.uint8))
rescaled_image.save("Output_int.png")

#### Example with 2 inpaintings (HLG)

In [ ]:
# prompt = 'Round Sunlight, strong high luminance'
# prompt = 'White Round Sun, Outdoor Environment map'
# prompt = "Dark Sky, low luminance, High Resolution, Outdoor Environment map"
prompt = "Remove Sun, Sky, no luminance, High Resolution, Outdoor Environment map"
negative_prompt = 'boxes, artifacts, light'

# image and mask_image should be PIL images.
#The mask structure is white for inpainting and black for keeping as is
# img, width, height, channels = load_exr("../data/abandoned_church_1k.exr")
img, width, height, channels = load_exr("./brick_factory_white.exr")
new_width, new_height = int(width * 1.0), int(height * 1.0)
resized_img = resize_exr(img, new_width, new_height)
resized_rgb_image = np.stack([resized_img['R'], resized_img['G'], resized_img['B']], axis=-1)
# rein_out = rgb_to_ext_reinhard(resized_rgb_image)
rein_out = rgb_to_hlg(resized_rgb_image)
# inp_img = Image.fromarray((rein_out * 255).clip(0, 255).astype(np.uint8))
# plt.imshow(inp_img)

mask_image = Image.open("mask_1_sky_large.png")
mask_image = pipe.mask_processor.blur(mask_image, blur_factor=10)
# plt.imshow(mask_image)

image: Image.Image = pipe(prompt=prompt, negative_prompt=negative_prompt, image=rein_out, mask_image=mask_image, num_inference_steps=50, strength=0.7, guidance_scale=7.5, width=1024, height=512, output_type='np').images[0]
plt.imshow(image)

prompt = "Sunlight, Natural, High Resolution, Outdoor Environment map"
negative_prompt = 'boxes, artifacts'
mask_image = Image.open("mask_2_light_large.png")
mask_image = pipe.mask_processor.blur(mask_image, blur_factor=10)
image: Image.Image = pipe(prompt=prompt, negative_prompt=negative_prompt, image=image, mask_image=mask_image, num_inference_steps=100, strength=0.7, guidance_scale=7.5, width=1024, height=512, output_type='np').images[0]

# rescaled_image = np.array(image, dtype = np.float32) / 255.0
save_exr(hlg_to_rgb(image), 'Output_int.exr')
rescaled_image = Image.fromarray((image * 255).clip(0, 255).astype(np.uint8))
rescaled_image.save("Output_int.png")

#### CHURCH

In [ ]:
# prompt = 'Round Sunlight, strong high luminance'
# prompt = 'White Round Sun, Outdoor Environment map'
prompt = "Clouds, low luminance, High Resolution, Outdoor Environment map"
negative_prompt = 'boxes, artifacts'

# image and mask_image should be PIL images.
#The mask structure is white for inpainting and black for keeping as is
img, width, height, channels = load_exr("./abandoned_church_1k_toinpv3.exr")
new_width, new_height = int(width * 1.0), int(height * 1.0)
resized_img = resize_exr(img, new_width, new_height)
resized_rgb_image = np.stack([resized_img['R'], resized_img['G'], resized_img['B']], axis=-1)
rein_out = rgb_to_ext_reinhard(resized_rgb_image)
inp_img = Image.fromarray((rein_out * 255).clip(0, 255).astype(np.uint8))
# plt.imshow(inp_img)

mask_image = Image.open("mask_2_fill_clouds.png")
mask_image = pipe.mask_processor.blur(mask_image, blur_factor=10)
# plt.imshow(mask_image)

image: Image.Image = pipe(prompt=prompt, negative_prompt=negative_prompt, image=inp_img, mask_image=mask_image, num_inference_steps=200, strength=0.5, guidance_scale=7.5, width=1024, height=512).images[0]

prompt = "Sunlight, Natural, High Resolution, Outdoor Environment map"
mask_image = Image.open("mask_1_fill_light.png")
mask_image = pipe.mask_processor.blur(mask_image, blur_factor=10)
image: Image.Image = pipe(prompt=prompt, negative_prompt=negative_prompt, image=image, mask_image=mask_image, num_inference_steps=200, strength=0.5, guidance_scale=7.5, width=1024, height=512).images[0]

rescaled_image = np.array(image, dtype = np.float32) / 255.0
save_exr(reinhard_to_rgb(rescaled_image, white_lum=100), 'Output_int.exr')
image.save("Output_int.png")

#### Example with only 1 inpainting

In [ ]:
# prompt = 'Round Sunlight, strong high luminance'
prompt = 'White Round Sun, Outdoor Environment map'
negative_prompt = 'boxes, artifacts'

# image and mask_image should be PIL images.
#The mask structure is white for inpainting and black for keeping as is
# img, width, height, channels = load_exr("../data/abandoned_church_1k.exr")
img, width, height, channels = load_exr("./abandoned_church_1k_toinp.exr")
new_width, new_height = int(width * 1.0), int(height * 1.0)
resized_img = resize_exr(img, new_width, new_height)
resized_rgb_image = np.stack([resized_img['R'], resized_img['G'], resized_img['B']], axis=-1)
rein_out = rgb_to_ext_reinhard(resized_rgb_image)
inp_img = Image.fromarray((rein_out * 255).clip(0, 255).astype(np.uint8))

mask_image = Image.open("mask.png")
mask_image = pipe.mask_processor.blur(mask_image, blur_factor=10)
plt.imshow(mask_image)

In [ ]:
image: Image.Image = pipe(prompt=prompt, negative_prompt=negative_prompt, image=inp_img, mask_image=mask_image, num_inference_steps=160, strength=0.5, guidance_scale=9.0, width=1024, height=512).images[0]
rescaled_image = np.array(image, dtype = np.float32) / 255.0
save_exr(reinhard_to_rgb(rescaled_image, white_lum=90), 'Output.exr')
image.save("Output.png")
plt.imshow(image)